In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emot --upgrade
!pip install emoji
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=5ace105fb68d1f23610d6334ec834dd2aacac312aaeb76698367230b651ad2d3
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-4-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-5-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
import pandas as pd

In [ ]:
bodo_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/train_BO_AH_HASOC2023.csv')
bodo_test=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/test_BO_AH_HASOC2023.csv')

In [ ]:
bodo_train['clean_text'] = bodo_train['text'].apply(lambda x:emo(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:remove_urls(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
bodo_train[bodo_train['clean_text'] =='**********************']

,S. No.,text,task_1,clean_text


In [ ]:
bodo_train['clean_text']

0                                                                                                 गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1                                                                                                                      निखावरि सुबुंफोरा सिखाव
2                                                                                                   मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3                                                                                                      थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4                                                      माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला
                                                                         ...                                                                  
1674                                                                   नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा

In [ ]:
bodo_test['clean_text'] = bodo_test['text'].apply(lambda x:emo(x))
bodo_test['clean_text'] = bodo_test['clean_text'].apply(lambda x:remove_urls(x))
bodo_test['clean_text'] = bodo_test['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
X_train=bodo_train['clean_text']
X_test=bodo_test['clean_text']
y_train=bodo_train['task_1']

In [ ]:
X_train.shape, X_test.shape

((1679,), (420,))

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [ ]:
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(X_train)
t.word_index['<PAD>'] = 0

In [ ]:
t.word_index

{'<UNK>': 1,
 'नों': 2,
 'आ': 3,
 'मा': 4,
 'सैमा': 5,
 'बे': 6,
 'नामा': 7,
 'आंग': 8,
 'बर': 9,
 'आं': 10,
 'नई': 11,
 'बियो': 12,
 'नोंनि': 13,
 'बोरमा': 14,
 'फुरकव': 15,
 'औ': 16,
 'साला': 17,
 'जामबा': 18,
 'फुरा': 19,
 'सासे': 20,
 'खामानि': 21,
 'बायदि': 22,
 'मानसी': 23,
 'लोदै': 24,
 'मोजां': 25,
 'हारिनि': 26,
 'फाग्ला': 27,
 'एै': 28,
 'माथो': 29,
 'नोंसोर': 30,
 'प्रमद': 31,
 'जों': 32,
 'नि': 33,
 'नवाओ': 34,
 'आरो': 35,
 'खव': 36,
 'जवंगनी': 37,
 'बुरबक': 38,
 'दं': 39,
 'गैया': 40,
 'बिमा': 41,
 'सोर': 42,
 'थाखाय': 43,
 'आव': 44,
 'मानो': 45,
 'दीनवी': 46,
 'दंग': 47,
 'बेखौ': 48,
 'बिदि': 49,
 'मानसि': 50,
 'जेबो': 51,
 'नोंलाय': 52,
 'बरनि': 53,
 'जोंनि': 54,
 'गुनदा': 55,
 'गाज्रि': 56,
 'मानसिखौ': 57,
 'ज्वंग': 58,
 'जानो': 59,
 'फुइसा': 60,
 'ना': 61,
 'नोंखौ': 62,
 'गैला': 63,
 'सिफा': 64,
 'खर': 65,
 'फुरखु': 66,
 'खौ': 67,
 'जाबाय': 68,
 'बेव': 69,
 'बै': 70,
 'निखावरि': 71,
 'सिखाव': 72,
 'बेलाय': 73,
 'जादों': 74,
 'बेनी': 75,
 'गावनि': 76,
 'था': 77,
 'मोसौ'

In [ ]:
max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']

(('मिलायखायो', 4797), ('<PAD>', 0), 1)

In [ ]:
train_sequences = t.texts_to_sequences(X_train)

In [ ]:
train_sequences

[[253, 21, 1502, 1503, 173, 457, 458, 870],
 [71, 300, 72],
 [4, 41, 65, 1504, 81, 871, 459, 78],
 [872, 18, 5, 17, 4, 1505, 79, 73],
 [221, 1506, 301, 2, 14, 162, 302, 460, 21, 303, 606, 82, 304, 873, 5, 17],
 [46, 461, 1507],
 [1508, 254, 1509, 1510],
 [361,
  462,
  14,
  162,
  362,
  9,
  17,
  874,
  26,
  81,
  200,
  174,
  875,
  255,
  1511,
  363,
  876,
  1512,
  877,
  607],
 [46, 37, 201, 36, 1513, 8],
 [608, 1514, 1515, 1516, 878, 609, 1517, 1518, 610],
 [364, 19, 611, 1519],
 [132, 222, 19, 1520],
 [146, 29, 612, 1521, 613, 463, 365, 223, 223, 1522, 83, 1523, 2, 18],
 [879, 36, 880, 8],
 [464, 11, 23, 66, 881, 95, 366, 614, 882],
 [13, 106, 1524, 305, 163, 306],
 [28,
  5,
  1525,
  1526,
  4,
  81,
  256,
  1527,
  7,
  302,
  460,
  107,
  367,
  53,
  1528,
  1529,
  38],
 [465, 3, 883, 11, 23, 66, 615],
 [884, 257, 885, 368, 616, 45, 9, 1530, 45, 224, 175],
 [58, 1531, 1532, 307, 120],
 [133, 3, 1533, 66, 34, 100, 202],
 [38, 369, 886, 203, 29, 1534, 39, 2, 617, 42,

In [ ]:
test_sequences = t.texts_to_sequences(X_test)

In [ ]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=4798
Number of Documents=1679


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens = [len(s) for s in train_sequences]
test_lens = [len(s) for s in test_sequences]

fig, ax = plt.subplots(1,2, figsize=(12, 6))
h1 = ax[0].hist(train_lens)
h2 = ax[1].hist(test_lens)

In [ ]:
MAX_SEQUENCE_LENGTH = 1000

In [ ]:
# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

((1679, 1000), (420, 1000))

In [ ]:
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0

In [ ]:
y_train = le.fit_transform(y_train)


In [ ]:
tt = [1,1,1, 0, 1]
print(le.inverse_transform(tt))

['NOT' 'NOT' 'NOT' 'HOF' 'NOT']


In [ ]:
VOCAB_SIZE1 = len(t.word_index)

In [ ]:
EMBED_SIZE = 300
EPOCHS=5
BATCH_SIZE=128

In [ ]:
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE1, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 300)         1439400   
                                                                 
 conv1d (Conv1D)             (None, 1000, 128)         153728    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 500, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 64)           32832     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 250, 64)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 250, 32)           8

In [ ]:
# Fit the model
model.fit(X_train, y_train,
          validation_split=0.1,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          verbose=1)

Epoch 1/5
12/12 [==============================] - 29s 2s/step - loss: 0.6773 - accuracy: 0.5930 - val_loss: 0.6703 - val_accuracy: 0.6071
Epoch 2/5
12/12 [==============================] - 31s 3s/step - loss: 0.6462 - accuracy: 0.5936 - val_loss: 0.5844 - val_accuracy: 0.7619
Epoch 3/5
12/12 [==============================] - 29s 2s/step - loss: 0.3499 - accuracy: 0.8729 - val_loss: 0.6902 - val_accuracy: 0.7976
Epoch 4/5
12/12 [==============================] - 27s 2s/step - loss: 0.0910 - accuracy: 0.9722 - val_loss: 0.7894 - val_accuracy: 0.8393
Epoch 5/5
12/12 [==============================] - 27s 2s/step - loss: 0.0171 - accuracy: 0.9940 - val_loss: 1.2023 - val_accuracy: 0.8274


In [ ]:
# # Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=1)
# print("Accuracy: %.2f%%" % (scores[1]*100))

97/97 [==============================] - 21s 219ms/step - loss: 0.6814 - accuracy: 0.6099
Accuracy: 60.99%


In [ ]:
# predictions = model.predict_classes(X_test1).ravel()
predictions = (model.predict(X_test) > 0.5).astype("int32")
# predictions[:10]

14/14 [==============================] - 2s 136ms/step


In [ ]:
predictions = ['HOF' if item == 1 else 'NOT' for item in predictions]
predictions[:10]

['HOF', 'HOF', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'HOF', 'HOF', 'NOT']

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=predictions, columns=['predictions'])
Submisssion_bodo = pd.DataFrame()
Submisssion_bodo['S. No.'] = bodo_test['S. No.']
Submisssion_bodo['task_1 '] = y_pred_df
Submisssion_bodo.to_csv('/content/drive/MyDrive/MUCS_test_bodo_run11.csv',index = None)

In [ ]:
# from sklearn.metrics import confusion_matrix, classification_report

# labels = ['NOT', 'HOF']
# print(classification_report(test_sentiments, predictions))
# pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

         HOF       0.44      0.20      0.28      1781
         NOT       0.37      0.65      0.47      1295

    accuracy                           0.39      3076
   macro avg       0.41      0.43      0.37      3076
weighted avg       0.41      0.39      0.36      3076



,NOT,HOF
NOT,360,1421
HOF,455,840
